**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Jan 22 13:24:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Assessment_one.zip (Unzipped Files)'
'Colab Notebooks'
'Copie de quizz du developpement durable du fun de dingue (1).gslides'
'Copie de quizz du developpement durable du fun de dingue.gslides'
 CV_Axel_Puech_COOK-E
 espagne.wav
 FILM
'instiut espagnol.docx'
'Interview Institut Culturel italien.docx'
'interview italie.rar'
'interview italie.wav'
'interview quebec.docx'
 Lords.of.Dogtown.2005.720p.BluRay.H264.AAC-RARBG.mp4
'My Drive'
'New Unity Project.zip'
'photo jeannette.zip'
'photo jeannette.zip (Unzipped Files)'
'popcorntime[1].apk'
 ProjectVHDL_M1.zip
 project_vhdl.zip
'Projet Capteur'
'projet transverse.rar'
 Projet_VHDL_S7.zip
'Projet VHDL.zip'
'Question institut hongrois.docx'
 sauvegarde
'site internet.7z'
 tesla.gdoc
 TestopenCv2.zip
 TextDetectionTesseract.zip
 tp_robot2.zip
'Untitled Diagram.drawio'
'wallonie Bruxelles (1).wav'
'wallonie Bruxelles (2).wav'
'wallonie Bruxelles.wav'
'War photography  Julien Bonnet  Axel Puech.gslides'
 yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14691 (delta 1), reused 4 (delta 0), pack-reused 14677
Receiving objects: 100% (14691/14691), 13.25 MiB | 24.05 MiB/s, done.
Resolving deltas: 100% (9985/9985), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Plaque" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-01-22 13:25:44--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  3.36MB/s    in 41s     

2021-01-22 13:26:26 (3.79 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
 extracting: data/obj/classes.txt    
  inflating: data/obj/images (1).jpg  
  inflating: data/obj/images (1).txt  
  inflating: data/obj/images (10).jpg  
  inflating: data/obj/images (10).txt  
  inflating: data/obj/images (11).jpg  
  inflating: data/obj/images (11).txt  
  inflating: data/obj/images (12).jpg  
  inflating: data/obj/images (12).txt  
  inflating: data/obj/images (13).jpg  
  inflating: data/obj/images (13).txt  
  inflating: data/obj/images (14).jpg  
  inflating: data/obj/images (14).txt  
  inflating: data/obj/images (15).jpg  
  inflating: data/obj/images (15).txt  
  inflating: data/obj/images (16).jpg  
  inflating: data/obj/images (16).txt  
  inflating: data/obj/images (17).jpg  
  inflating: data/obj/images (17).txt  
  inflating: data/obj/images (18).jpg  
  inflating: data/obj/images (18).txt  
  inflating: data/obj/images (19).jpg  
  inflating: data/obj/images (19).txt  
  inflating: data/obj/images (2).jpg  
  inflat

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.530075 0.648148 0.180451 0.058201

['0', '0.530075', '0.648148', '0.180451', '0.058201']
0 data/obj/images (40).txt
0 0.530075 0.648148 0.180451 0.058201
0 0.492593 0.639037 0.200000 0.080214

['0', '0.492593', '0.639037', '0.200000', '0.080214']
1 data/obj/images (71).txt
0 0.492593 0.639037 0.200000 0.080214
0 0.213504 0.690217 0.193431 0.108696

['0', '0.213504', '0.690217', '0.193431', '0.108696']
2 data/obj/images (19).txt
0 0.213504 0.690217 0.193431 0.108696
0 0.326786 0.775000 0.296429 0.150000

['0', '0.326786', '0.775000', '0.296429', '0.150000']
3 data/obj/images (17).txt
0 0.326786 0.775000 0.296429 0.150000
0 0.477459 0.567961 0.176230 0.058252

['0', '0.477459', '0.567961', '0.176230', '0.058252']
4 data/obj/images (55).txt
0 0.477459 0.567961 0.176230 0.058252
0 0.731557 0.689320 0.323770 0.087379

['0', '0.731557', '0.689320', '0.323770', '0.087379']
5 data/obj/images (80).txt
0 0.731557 0.689320 0.323770 0.087379
0 0.482258 0.530675 0.196774 0.104294

['0', '0.4822

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/images (49).jpg', 'data/obj/images (54).jpg', 'data/obj/images (82).jpg', 'data/obj/images (76).jpg', 'data/obj/images (6).jpg', 'data/obj/images (71).jpg', 'data/obj/images (37).jpg', 'data/obj/images (16).jpg', 'data/obj/images (44).jpg', 'data/obj/images (19).jpg', 'data/obj/images (55).jpg', 'data/obj/images (20).jpg', 'data/obj/images (38).jpg', 'data/obj/images (11).jpg', 'data/obj/images (25).jpg', 'data/obj/images (4).jpg', 'data/obj/images (10).jpg', 'data/obj/images (8).jpg', 'data/obj/images (72).jpg', 'data/obj/images (22).jpg', 'data/obj/images (24).jpg', 'data/obj/images (26).jpg', 'data/obj/images (78).jpg', 'data/obj/images (52).jpg', 'data/obj/images (50).jpg', 'data/obj/images (32).jpg', 'data/obj/images (63).jpg', 'data/obj/images (9).jpg', 'data/obj/images (73).jpg', 'data/obj/images (12).jpg', 'data/obj/images (85).jpg', 'data/obj/images (69).jpg', 'data/obj/images (5).jpg', 'data/obj/images (46).jpg', 'data/obj/images (80).jpg', 'data/obj/images (66).jp

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.609604), count: 1, class_loss = 0.144005, iou_loss = 0.091500, total_loss = 0.235504 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.770735), count: 3, class_loss = 0.269664, iou_loss = 0.093340, total_loss = 0.363004 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000009, iou_loss = 0.000000, total_loss = 0.000009 
 total_bbox = 63568, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000104, iou_loss = 0.000000, total_loss = 0.000104 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.795699), count: 4, class_loss = 0.045587, iou_loss = 0.098621, total_loss = 0.144208 
v3 (mse loss, Normalizer: (iou: